# Tracking Sensor Bias

We want to compute the joint posterior over sensors' biases in a 2-D tracking setting.

In [ ]:
from collections import OrderedDict

import torch
from torch.optim import Adam

import pyro
import pyro.distributions as dist

import funsor
import funsor.pyro
import funsor.distributions as f_dist
import funsor.ops as ops
from funsor.pyro.convert import dist_to_funsor, mvn_to_funsor, matrix_and_mvn_to_funsor, tensor_to_funsor
from funsor.interpreter import interpretation, reinterpret
from funsor.optimizer import apply_optimizer
from funsor.terms import lazy
from funsor.domains import bint, reals
from funsor.sum_product import sequential_sum_product

Simulate some synthetic data:

In [ ]:
num_sensors = 5
num_frames = 100

# simulate biased sensors
sensors  = []
for _ in range(num_sensors):
    true_pos = 10 * torch.rand(2)  # in a box
    biased_pos = true_pos + torch.randn(2)
    sensors.append({
        "true_pos": true_pos,
        "biased_pos": biased_pos,
    })

# simulate a single track
track = []
z = 10 * torch.rand(2)  # initial state
for t in range(num_frames):
    # Advance latent state.
    z += torch.randn(2)
    z.clamp_(min=0, max=10)  # keep in the box
    
    # Observe via a random sensor.
    sensor_id = pyro.sample('id', dist.Categorical(torch.ones(num_sensors)))
    x = z - sensors[sensor_id]["biased_pos"]
    track.append({"sensor_id": sensor_id, "x": x})

Now let's set up a tracking problem in Funsor. We start by modeling the biases of each sensor.

In [ ]:
%pdb off

In [ ]:
# bias = funsor.pyro.convert.mvn_to_funsor(
#     dist.MultivariateNormal(
#         torch.zeros(num_sensors, 2),
#         torch.eye(2, requires_grad=True)  # This can be learned
#     ),
#     event_dims=("biased_pos",),
#     real_inputs=OrderedDict([("biased_pos", reals(2))])
# )(value="bias")

bias_cov = torch.eye(2, requires_grad=True)  # This can be learned
# Instead create a giant joint Gaussian:
bias = sum(
    funsor.pyro.convert.mvn_to_funsor(
        dist.MultivariateNormal(
            torch.zeros(2),
            cov
        ),
        event_dims=(("bias_{}".format(j) for j in range(num_sensors)),)
    )(value="bias_{}".format(i))
    for i in range(num_sensors)
)

Set up the filter in funsor.

In [ ]:
def model(track):
    # Similar to funsor.pyro.hmm.GaussianHMM.__init__()
    init_dist = torch.distributions.MultivariateNormal(torch.zeros(2), torch.eye(2))
    # this can be parameterized by a lower dimensional vector 
    # to learn a structured transition matrix
    # eg a GP with a matern v=3/2 kernel
    transition_matrix = torch.randn(2, 2, requires_grad=True)

    transition_dist = torch.distributions.MultivariateNormal(
        torch.zeros(2),
        torch.eye(2))
    observation_matrix = torch.randn(2, 2)
    observation_dist = torch.distributions.MultivariateNormal(
        torch.zeros(2),
        torch.eye(2))

    init = dist_to_funsor(init_dist)(value="state")
    trans = matrix_and_mvn_to_funsor(transition_matrix, transition_dist,
                                     ("time",), "state", "state(time=1)")
    obs = matrix_and_mvn_to_funsor(observation_matrix, observation_dist,
                                   ("time",), "state(time=1)", "value")
    # Now this is the crux, we add bias to the observation
    sensor_ids = funsor.torch.Tensor(
        torch.tensor([frame["sensor_id"] for frame in track]),
        OrderedDict([("sensor_id", bint(num_frames))]),
        dtype=len(sensors)
    )
    biased_observations = funsor.torch.Tensor(
        torch.stack([frame["x"] for frame in track]),
        OrderedDict([("sensor_id", bint(num_frames))])
    )
    bias_over_time = bias(sensor_id=sensor_ids)
    obs = obs(value=biased_observations)

    # Similar to funsor.pyro.hmm.GaussianHMM.log_prob()
    # ndims = max(len(batch_shape), value.dim() - event_dim)
    # value = tensor_to_funsor(value, ("time",), event_output=event_dim - 1,
    #                          dtype=self.dtype)

    # obs = obs(value=value)
    result = trans + obs

    result = sequential_sum_product(ops.logaddexp, ops.add,
                                    result, "time", {"state": "state(time=1)"})
    result += init
    result = result.reduce(ops.logaddexp, frozenset(["state", "state(time=1)"]))

## Inference

Finally we have a result that is a joint Gaussian over the biases.
We can
1. optimize all parameters to maximize `result.reduce(obs.logaddexp)`
2. estimate the joint distribution over all bias parameters.

In [ ]:
num_epochs = 200
params = [bias_cov, transition_matrix]
optim = Adam(params, lr=1e-3)
for i in range(num_epochs):
    optim.zero_grad()
    with interpretation(lazy):
        log_prob = apply_optimizer(model(track))
    loss = -reinterpret(log_prob).data
    loss.backward()
    if i % 10 == 0:
        print(loss)
    optim.step()
print(params)

Visualize the joint posterior distribution.